In [1]:
# execute to import notebook styling for tables and width etc.
from IPython.core.display import HTML
import urllib.request
response = urllib.request.urlopen('https://raw.githubusercontent.com/DataScienceUWL/DS775v2/master/ds755.css')
HTML(response.read().decode("utf-8"));

<font size=18>Lesson 02 - Self-Assessment Solutions</font>

# <font color = "blue"> Self Assessment: Positive Shadow Price </font>

Answer: True

# <font color = "blue"> Self Assessment: Allowable Range (Objective Coef) </font>

Answer: True

# <font color = "blue"> Self Assessment: Changing Parameters </font>

Answer: False

# <font color = "blue"> Self Assessment: Graphical Exploration of Sensitivity </font>

(a) Optimal $Z=22$ occurs when $x=6$ and $y=2$.

<img src="images/HW_exploration_a.png" width="640" height="450"> 

(b) Shadow price $=23-22=1$. The new optimal is $Z=23$ with coordinates $x=4$ and $y=3$.

<img src="images/HW_exploration_b.png" width="640" height="450"> 

(c) The allowable range for resource 2 is $10 \leq b_2 \leq 15$. 

The lower bound is 10, as shown here.

<img src="images/HW_exploration_c2.png" width="640" height="450"> 

The upper bound is 15, as shown here.

<img src="images/HW_exploration_c1.png" width="640" height="450"> 

(d) The allowable range for the unit profit of activity 2 is $4 \leq c_2 \leq 6$. 

The lower bound is 4, as shown here.

<img src="images/HW_exploration_d2.png" width="640" height="450"> 

The upper bound is 6, as shown here.

<img src="images/HW_exploration_d1.png" width="640" height="450"> 

# <font color = "blue"> Self-Assessment: Solve and Perform Sensitivity </font>

(c) See the code in the following cells and the associated output.

(b) From the column labeled "Marginal" in the top table of the GLPK sensitivity report below, the shadow prices are 0.667 for resource 1 and 1 for resource 2.

In [14]:
# Code for textbook problem 4.7-6(c)

from pyomo.environ import *

# Concret Model
model = ConcreteModel(name = "Generic")

# Decision Variables 
model.x = Var( ['x1','x2','x3','x4'], 
              domain = NonNegativeReals)

# Objective 
model.obj = Objective( expr = 5*model.x['x1'] + 4*model.x['x2'] - model.x['x3'] +
                      3*model.x['x4'], sense = maximize)

# Constraints
model.Constraint1 = Constraint( expr = 3*model.x['x1'] + 2*model.x['x2'] - 
                               3*model.x['x3'] + model.x['x4'] <= 24 )
model.Constraint2 = Constraint( expr = 3*model.x['x1'] + 3*model.x['x2'] + 
                               model.x['x3'] + 3*model.x['x4'] <= 36 )
                      
# Solve
solver = SolverFactory('glpk')
solver.solve(model)

# remove the comment symbol to see the pyomo display of results
# display(model)

# print a shorter summary of relevant results
print(" Z = ", model.obj())
print("x1 = ",model.x["x1"]())
print("x2 = ",model.x["x2"]())
print("x3 = ",model.x["x3"]())
print("x4 = ",model.x["x4"]())


 Z =  52.0
x1 =  11.0
x2 =  0.0
x3 =  3.0
x4 =  0.0


In [15]:
# write the model to a sensitivity report
model.write('model.lp', io_options={'symbolic_solver_labels': True})
!glpsol -m model.lp --lp --ranges sensit.sen

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 -m model.lp --lp --ranges sensit.sen
Reading problem data from 'model.lp'...
3 rows, 5 columns, 9 non-zeros
34 lines were read
GLPK Simplex Optimizer, v4.65
3 rows, 5 columns, 9 non-zeros
Preprocessing...
2 rows, 4 columns, 8 non-zeros
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  3.000e+00  ratio =  3.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 2
*     0: obj =  -0.000000000e+00 inf =   0.000e+00 (3)
*     2: obj =   5.200000000e+01 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0.0 Mb (40412 bytes)
Write sensitivity analysis report to 'sensit.sen'...


In [16]:
# widen browser and/or close TOC to see sensitivity report
import numpy as np
np.set_printoptions(linewidth=110)
f = open('sensit.sen', 'r')
file_contents = f.read()
print(file_contents)
f.close()

GLPK 4.65 - SENSITIVITY ANALYSIS REPORT                                                                         Page   1

Problem:    
Objective:  obj = 52 (MAXimum)

   No. Row name     St      Activity         Slack   Lower bound       Activity      Obj coef  Obj value at Limiting
                                          Marginal   Upper bound          range         range   break point variable
------ ------------ -- ------------- ------------- -------------  ------------- ------------- ------------- ------------
     1 c_u_Constraint1_
                    NU      24.00000        .               -Inf     -108.00000       -.66667     -36.00000 x(x1)
                                            .66667      24.00000       36.00000          +Inf      60.00000 x(x3)

     2 c_u_Constraint2_
                    NU      36.00000        .               -Inf       24.00000      -1.00000      40.00000 x(x3)
                                           1.00000      36.00000           +Inf        

# <font color = "blue"> Self-Assessment: Formulate, Solve, and Perform Sensitivity #1 </font>

(a) See the code and output for the cell below.  The maximum profit is \\$3500, obtained when 2000 toys and 1000 subassemlies are produced per day.

(f) From the "Obj coef range" in the bottom table of the GLPK sensitivty report, the allowable range of the unit profit for toys is \\$2.50 to \\$5 whereas that for subassemblies is -\\$3 to -\\$1.50.

In [26]:
# Code for textbook problem 7.3-4 (a & f)

from pyomo.environ import *

# Concret Model
model = ConcreteModel(name = "TannerCo")

# Decision Variables 
model.x = Var( ['x1_toys','x2_subs'], 
              domain = NonNegativeReals)

# Objective 
model.obj = Objective( expr = 3*model.x['x1_toys'] - 2.5*model.x['x2_subs'], sense = maximize)

# Constraints
model.Constraint1 = Constraint( expr = 2*model.x['x1_toys'] - model.x['x2_subs'] <= 3000 )
model.Constraint2 = Constraint( expr = model.x['x1_toys'] - model.x['x2_subs'] <= 1000 )
                      
# Solve
solver = SolverFactory('glpk')
solver.solve(model)

# remove the comment symbol to see the pyomo display of results
# display(model)

# print a shorter summary of relevant results
import babel.numbers as numbers   # needed to display as currency
print("Total Profit = ", numbers.format_currency(model.obj(),'USD', locale='en_US'))
print("Daily Production rate of Toys:         ",model.x["x1_toys"]())
print("Daily Production rate of Subassemblies:",model.x["x2_subs"]())


Total Profit =  $3,500.00
Daily Production rate of Toys:          2000.0
Daily Production rate of Subassemblies: 1000.0


In [24]:
# write the model to a sensitivity report
model.write('model.lp', io_options={'symbolic_solver_labels': True})
!glpsol -m model.lp --lp --ranges sensit.sen

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 -m model.lp --lp --ranges sensit.sen
Reading problem data from 'model.lp'...
3 rows, 3 columns, 5 non-zeros
26 lines were read
GLPK Simplex Optimizer, v4.65
3 rows, 3 columns, 5 non-zeros
Preprocessing...
2 rows, 2 columns, 4 non-zeros
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  2.000e+00  ratio =  2.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 2
*     0: obj =  -0.000000000e+00 inf =   0.000e+00 (1)
*     2: obj =   3.500000000e+03 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0.0 Mb (40412 bytes)
Write sensitivity analysis report to 'sensit.sen'...


In [25]:
# widen browser and/or close TOC to see sensitivity report
import numpy as np
np.set_printoptions(linewidth=110)
f = open('sensit.sen', 'r')
file_contents = f.read()
print(file_contents)
f.close()

GLPK 4.65 - SENSITIVITY ANALYSIS REPORT                                                                         Page   1

Problem:    
Objective:  obj = 3500 (MAXimum)

   No. Row name     St      Activity         Slack   Lower bound       Activity      Obj coef  Obj value at Limiting
                                          Marginal   Upper bound          range         range   break point variable
------ ------------ -- ------------- ------------- -------------  ------------- ------------- ------------- ------------
     1 c_u_Constraint1_
                    NU    3000.00000        .               -Inf     2000.00000       -.50000    3000.00000 x(x2_subs)
                                            .50000    3000.00000           +Inf          +Inf          +Inf

     2 c_u_Constraint2_
                    NU    1000.00000        .               -Inf           -Inf      -2.00000          -Inf
                                           2.00000    1000.00000     1500.00000          +In

# <font color = "blue"> Self-Assessment: Formulate, Solve, and Perform Sensitivity #2 </font>

(a) This is pretty simple.  Add an extra constraint that $x_1 \leq 2500$.  The maximum profit is still \\$3500, obtained when 2000 toys and 1000 subassemlies are produced per day.  See the code cell below.

See cells befow for parts b and f.

In [27]:
# Code for textbook problem 7.3-5 (a)

from pyomo.environ import *

# Concret Model
model = ConcreteModel(name = "TannerCo")

# Decision Variables 
model.x = Var( ['x1_toys','x2_subs'], 
              domain = NonNegativeReals)

# Objective 
model.obj = Objective( expr = 3*model.x['x1_toys'] - 2.5*model.x['x2_subs'], sense = maximize)

# Constraints
model.Constraint1 = Constraint( expr = 2*model.x['x1_toys'] - model.x['x2_subs'] <= 3000 )
model.Constraint2 = Constraint( expr = model.x['x1_toys'] - model.x['x2_subs'] <= 1000 )
model.Constraint3 = Constraint( expr = model.x['x1_toys'] <= 2500 )

                      
# Solve
solver = SolverFactory('glpk')
solver.solve(model)

# remove the comment symbol to see the pyomo display of results
# display(model)

# print a shorter summary of relevant results
import babel.numbers as numbers   # needed to display as currency
print("Total Profit = ", numbers.format_currency(model.obj(),'USD', locale='en_US'))
print("Daily Production rate of Toys:         ",model.x["x1_toys"]())
print("Daily Production rate of Subassemblies:",model.x["x2_subs"]())

Total Profit =  $3,500.00
Daily Production rate of Toys:          2000.0
Daily Production rate of Subassemblies: 1000.0


(b) Run the model with RHS of coefficient 1 at 3001 rather than 3000. The shadow price for subassembly A is $0.50, which is the maximum premium that the company should be willing to pay.  See the code in the next cell.

In [28]:
# Code for textbook problem 7.3-5 (b)

from pyomo.environ import *

# Concret Model
model = ConcreteModel(name = "TannerCo")

# Decision Variables 
model.x = Var( ['x1_toys','x2_subs'], 
              domain = NonNegativeReals)

# Objective 
model.obj = Objective( expr = 3*model.x['x1_toys'] - 2.5*model.x['x2_subs'], sense = maximize)

# Constraints
model.Constraint1 = Constraint( expr = 2*model.x['x1_toys'] - model.x['x2_subs'] <= 3001 )
model.Constraint2 = Constraint( expr = model.x['x1_toys'] - model.x['x2_subs'] <= 1000 )
model.Constraint3 = Constraint( expr = model.x['x1_toys'] <= 2500 )

                      
# Solve
solver = SolverFactory('glpk')
solver.solve(model)

# remove the comment symbol to see the pyomo display of results
# display(model)

# print a shorter summary of relevant results
import babel.numbers as numbers   # needed to display as currency
print("Total Profit = ", numbers.format_currency(model.obj(),'USD', locale='en_US'))
print("Daily Production rate of Toys:         ",model.x["x1_toys"]())
print("Daily Production rate of Subassemblies:",model.x["x2_subs"]())

Total Profit =  $3,500.50
Daily Production rate of Toys:          2001.0
Daily Production rate of Subassemblies: 1001.0


(f) As shown in the sensitivity report, the shadow price is $0.50 for subassembly A and $2 for subassembly B. According to the activity range, the allowable range for the right-hand side of the subassembly A constraint is 2,000 to 3,500. The allowable range for the right-hand side of the subassembly B constraint is 500 to 1,500.

In [3]:
# Code for textbook problem 7.3-5 (f)

from pyomo.environ import *

# Concret Model
model = ConcreteModel(name = "TannerCo")

# Decision Variables 
model.x = Var( ['x1_toys','x2_subs'], 
              domain = NonNegativeReals)

# Objective 
model.obj = Objective( expr = 3*model.x['x1_toys'] - 2.5*model.x['x2_subs'], sense = maximize)

# Constraints
model.Constraint1 = Constraint( expr = 2*model.x['x1_toys'] - model.x['x2_subs'] <= 3000 )
model.Constraint2 = Constraint( expr = model.x['x1_toys'] - model.x['x2_subs'] <= 1000 )
model.Constraint3 = Constraint( expr = model.x['x1_toys'] <= 2500 )

                      
# Solve
solver = SolverFactory('glpk')
solver.solve(model)

# write the model to a sensitivity report
model.write('model.lp', io_options={'symbolic_solver_labels': True})
!glpsol -m model.lp --lp --ranges sensit.sen

# widen browser and/or close TOC to see sensitivity report
import numpy as np
np.set_printoptions(linewidth=110)
f = open('sensit.sen', 'r')
file_contents = f.read()
print(file_contents)
f.close()

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 -m model.lp --lp --ranges sensit.sen
Reading problem data from 'model.lp'...
4 rows, 3 columns, 6 non-zeros
30 lines were read
GLPK Simplex Optimizer, v4.65
4 rows, 3 columns, 6 non-zeros
Preprocessing...
2 rows, 2 columns, 4 non-zeros
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  2.000e+00  ratio =  2.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 2
*     0: obj =  -0.000000000e+00 inf =   0.000e+00 (1)
*     2: obj =   3.500000000e+03 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0.0 Mb (40412 bytes)
Write sensitivity analysis report to 'sensit.sen'...
GLPK 4.65 - SENSITIVITY ANALYSIS REPORT                                                                         Page   1

Problem:    
Objective:  obj = 3500 (MAXimum)

   No. Row name     St      Activity         Slack   Lower bound       Activity      Obj coef  O

# <font color="blue">Self Assessment: Investment Allocation </font>

In [6]:
# Unfold for code
from pyomo.environ import *
invests = ['Friend1', 'Friend2']
constraints = ['Fraction1','Fraction2','Money','Work_Hours']
profit_rate = {'Friend1': 9000, 'Friend2': 9000}
constraint_rhs = {'Fraction1':1,'Fraction2':1,'Money':12000,'Work_Hours':600}
constraint_coef = {
    'Fraction1': {
        'Friend1': 1,
        'Friend2': 0
    },
    'Fraction2': {
        'Friend1': 0,
        'Friend2': 1
    },
    'Money': {
        'Friend1': 10000,
        'Friend2': 8000
    },
    'Work_Hours': {
        'Friend1': 400,
        'Friend2': 500
    },
}

#Concrete Model
model = ConcreteModel()

#Decision Variables
model.invest_frac = Var(invests, domain=NonNegativeReals)

#Objective
model.profit = Objective(expr=sum(profit_rate[i] * model.invest_frac[i]
                               for i in invests),
                      sense=maximize)

model.constraints = ConstraintList()
for c in constraints:
    model.constraints.add(
        sum(constraint_coef[c][i] * model.invest_frac[i]
            for i in invests) <= constraint_rhs[c])

# Solve
solver = SolverFactory('glpk')
solver.solve(model)

# display solution
import babel.numbers as numbers  # needed to display as currency
print("Maximum Profit = ",
      numbers.format_currency( model.profit(), 'USD', locale='en_US'))

for i in invests:
    print("Batches of " + i + " = {}".format(model.invest_frac[i]()))

Maximum Profit =  $12,000.00
Batches of Friend1 = 0.666666666666667
Batches of Friend2 = 0.666666666666666


# <font color="blue">Self Assessment: A Holiday Factory </font>

In [7]:
# unfold to see Pyomo solution
from pyomo.environ import *

# Concrete Model
model = ConcreteModel(name="HolidayFactory")

# Parameters and Index Sets
num_months = 11
max_duration = 11

months = range(1, num_months + 1)
durations = range(1, max_duration + 1)

rate = [sum(20 - i for i in range(month)) for month in range(1, 12)]
rent = dict(zip(durations, rate))
space = dict(
    zip(months,
        [2000, 2000, 3000, 4000, 6000, 10000, 10000, 10000, 9000, 7000, 5000]))

In [9]:
# unfold to see Pyomo solution
from pyomo.environ import *

# Concrete Model
model = ConcreteModel(name="HolidayFactory")

# Parameters and Index Sets
num_months = 11
max_duration = 11

months = range(1, num_months + 1)
durations = range(1, max_duration + 1)

rate = [sum(20 - i for i in range(month)) for month in range(1, 12)]
rent = dict(zip(durations, rate))
space = dict(
    zip(months,
        [2000, 2000, 3000, 4000, 6000, 10000, 10000, 10000, 9000, 7000, 5000]))

# Decision Variables
model.x_sqft = Var(months, durations, domain=NonNegativeReals)

# Objective
model.obj = Objective(expr=sum(rent[d] * model.x_sqft[m, d] for m in months
                               for d in durations))


model.display()
# # Constraints
# model.space_ct = ConstraintList()
# for month in months:
#     model.space_ct.add(
#         sum(model.x_sqft[m, d] for m in months for d in durations
#             if m <= month and m + d > month) >= space[month])

# model.time_rule_ct = ConstraintList()
# for m in months:
#     for d in durations:
#         if m + d > num_months + 1:
#             model.time_rule_ct.add(model.x_sqft[m, d] == 0)

# # Solve
# solver = SolverFactory('glpk')
# solver.solve(model)

# # print a short summary of relevant results
# import babel.numbers as numbers  # needed to display as currency
# print("Total Cost = ",
#       numbers.format_currency(model.obj(), 'USD', locale='en_US'))

# print("\nHere are the amounts to lease by month and duration:")
# for m in months:
#     for d in durations:
#         if model.x_sqft[m, d]() > 0:
#             print("Lease {:.0f}".format(model.x_sqft[m, d]()) +
#                   " sq ft in month {:d}".format(m) +
#                   " for {:d} months".format(d))

# print("\nHere are the amounts needed and the total amount needed in each month:")
# for m in months:
#     print("In month {:d}".format(m) +
#           ", {:d} square feet are needed".format(space[m]) +
#           " and {} square feet are leased".format(
#               sum(model.x_sqft[i, d]() for d in durations
#                   for i in months if i <= m and i + d > m)))

Model HolidayFactory

  Variables:
    x_sqft : Size=121, Index=x_sqft_index
        Key      : Lower : Value : Upper : Fixed : Stale : Domain
          (1, 1) :     0 :  None :  None : False :  True : NonNegativeReals
          (1, 2) :     0 :  None :  None : False :  True : NonNegativeReals
          (1, 3) :     0 :  None :  None : False :  True : NonNegativeReals
          (1, 4) :     0 :  None :  None : False :  True : NonNegativeReals
          (1, 5) :     0 :  None :  None : False :  True : NonNegativeReals
          (1, 6) :     0 :  None :  None : False :  True : NonNegativeReals
          (1, 7) :     0 :  None :  None : False :  True : NonNegativeReals
          (1, 8) :     0 :  None :  None : False :  True : NonNegativeReals
          (1, 9) :     0 :  None :  None : False :  True : NonNegativeReals
         (1, 10) :     0 :  None :  None : False :  True : NonNegativeReals
         (1, 11) :     0 :  None :  None : False :  True : NonNegativeReals
          (2, 1) :   

# <font color="blue">Self Assessment: Supply and Demand Problem </font>

Let $F$ be the set of factories and let $C$ be the set of customers.

Decision Variables:  let $x_{f,c}$ be the number of units shipped from factory $f \in F$ to customer $c \in C$

Constants:  
- $q_{f,c}$ is the shipping cost per unit between factory $f \in F$ and customer $c \in C$
- $d_c$ is the number of units demanded by customer $c \in C$
- $s_f$ is the number of units supplied by factory $f \in F$

Objective Function:  minimize $ Cost = \displaystyle \sum_{f \in F} \sum_{c \in C} q_{f,c} x_{f,c}$

Constraints:
- Supply: $ \displaystyle \sum_{c \in C} x_{f,c} = s_f, \mbox{ for each } f \in F$
- Demand: $ \displaystyle \sum_{f \in F} x_{f,c} = d_c, \mbox{ for each } c \in C$
- Nonnegativity: $x_{f,c} \geq 0$ for each $f \in F, c \in C$

In [13]:
# unfold to see Pyomo solution
from pyomo.environ import *

factories = ['factory1', 'factory2']
supply = dict(zip(factories, [400, 500]))

customers = ['cust1', 'cust2', 'cust3']
demand = dict(zip(customers, [300, 200, 400]))

usc = [[600, 800, 700], [400, 900, 600]]
unit_ship_cost = {
    factories[f]: {customers[c]: usc[f][c]
                   for c in range(len(customers))}
    for f in range(len(factories))
}

# throw an error if total supply and demand do not match
assert (sum(supply.values()) == sum(demand.values()))

from pyomo.environ import *

model = ConcreteModel()

model.transp = Var(factories, customers, domain=NonNegativeReals)


model.total_cost = Objective(expr=sum(unit_ship_cost[f][c] * model.transp[f, c]
                                      for f in factories for c in customers),
                             sense=minimize)

model.supply_ct = ConstraintList()
for f in factories:
    model.supply_ct.add(
        sum(model.transp[f, c] for c in customers) == supply[f])

model.demand_ct = ConstraintList()
for c in customers:
    model.demand_ct.add(
        sum(model.transp[f, c] for f in factories) == demand[c])

# solve and display
solver = SolverFactory('glpk')
solver.solve(model)

# display solution
import babel.numbers as numbers  # needed to display as currency
print("Minimum Total Cost = ",
      numbers.format_currency(model.total_cost(), 'USD', locale='en_US'))
# put amounts in dataframe for nicer display
import pandas as pd
dvars = pd.DataFrame([[model.transp[f, c]() for c in customers]
                      for f in factories],
                     index=factories,
                     columns=customers)
print("Number to ship from each factory to each customer:")
dvars

Minimum Total Cost =  $540,000.00
Number to ship from each factory to each customer:
demand_ct : Size=3, Index=demand_ct_index, Active=True
    Key : Lower : Body                                            : Upper : Active
      1 : 300.0 : transp[factory1,cust1] + transp[factory2,cust1] : 300.0 :   True
      2 : 200.0 : transp[factory1,cust2] + transp[factory2,cust2] : 200.0 :   True
      3 : 400.0 : transp[factory1,cust3] + transp[factory2,cust3] : 400.0 :   True
